In [102]:
import tweepy
import re

In [2]:
import urllib3
import urllib3.request

import csv
import json
from pandas.io.json import json_normalize

import pandas as pd
import numpy as np

In [3]:
pd.options.display.max_colwidth = 1000


In [4]:
####input your credentials here
consumer_key = '4ha4rLgP6Ci6fEZtaqttGTKoA'
consumer_secret = '5ckLaCgfTdfmWM7qS9f2w05pDCSIWRCTHlm7RLnKwK9tCWIz9P'
access_token = '602145669-jHmxtsl0wSZDFeZxi81GcTzYrD87dRBhF78ip0qo'
access_token_secret = 'YFLMmVVdcN4gb4KDX3MeOjbjxoKnnsFvjKxjRGMkkEZ5D'

In [5]:
input_hashtag = "giletjaune"
n_tweets = 1
language = "fr"
since_date = "2018-03-07"

In [7]:
####request to Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify = True)

inc = 0
for tweet in tweepy.Cursor(api.search,q="#"+input_hashtag,#count=n_tweets,
                           lang=language,
                           since=since_date, tweet_mode='extended').items():
    inc = +1
    with open('tweets/tweet'+str(tweet.id)+'.json', 'w', encoding='utf8') as file:
        ####you need to create a 'tweets' folder
        
        tweet_json = tweet._json
        json.dump(tweet_json, file)
        

Rate limit reached. Sleeping for: 381


KeyboardInterrupt: 

In [8]:
def is_a_retweet(tweet):
    #### returns a boolean 
    return tweet.get('retweeted_status',None) != None

In [103]:
#initialisation of databases, df files

d_users = {'user_id': [],
           'name':[],
           'screen_name':[],
           'location':[],
           'description':[],
           'url':[],
           'followers_count':[],
           'friends_count':[],
           'listed_count':[],
           'created_at':[],
           'favourites_count':[],
           'geo_enabled':[],
           'verified':[],
           'statuses_count':[],
           'lang':[],
           'contributors_enabled':[]
          }
df_users = pd.DataFrame(data=d_users)

d_tweets = {'tweet_id': [],
            'created_at': [],
            'text': [],
            'truncated': [],
            'source': [],
            'in_reply_to_status_id': [],
            'in_reply_to_user_id': [],
            'in_reply_to_screen_name': [],
            'user_id': [],
            'geo': [],
            'coordinates': [],
            'place': [],
            'contributors': [],
            'is_quote_status': [],
            'retweet_count': [],
            'favorite_count': [],
            'favorited': [],
            'retweeted': [],
            'lang': []         
           }
df_tweets = pd.DataFrame(data=d_tweets)


d_hashtags={'hashtag': [], 'tweet_id': []}
df_hashtags = pd.DataFrame(data=d_hashtags)

d_users_mentions = {'tweet_id':[], 'user_id': [], 'screen_name':[], 'name':[]}
df_users_mentions = pd.DataFrame(data=d_users_mentions)

d_retweet_users = {'user_id':[], 'original_user_id':[], 'original_tweet_id':[]}
df_retweet_users= pd.DataFrame(data=d_retweet_users)

d_influent_users = {'user_id':[], 
                    'name':[],
                    'retweet_count':[],
                    'favorite_count':[], 
                    'description':[] 
                   }

df_influent_users = pd.DataFrame(data=d_influent_users)



In [104]:
def store_hashtag(tweet):
    global input_hashtag
    global df_hashtags
    if tweet['entities']['hashtags']==[]:
        df_hashtags = df_hashtags.append({'hashtag': input_hashtag ,'tweet_id':tweet['id_str']}, ignore_index=True)
    else:
        for raw_hash in tweet['entities']['hashtags']:
            df_hashtags = df_hashtags.append({'hashtag': raw_hash['text'] ,'tweet_id':tweet['id_str']}, ignore_index=True)

def store_user_mentions(tweet):
    global df_users_mentions
    tweet_id = tweet['id_str']
    for raw_mention in tweet['entities']['user_mentions']:
        screen_name = raw_mention['screen_name']
        name = raw_mention['name']
        user_id = raw_mention['id_str']
        df_users_mentions = df_users_mentions.append({'tweet_id':tweet_id, 'user_id': user_id, 'screen_name':screen_name, 'name':name},ignore_index=True)

def store_users(tweet):
    global df_users
    user = tweet['user']
    if user['id_str'] not in df_users['user_id'].values:
        
        d_user = {'user_id': user['id_str'],
                  'name':user['name'],
                  'screen_name':user['screen_name'],
                  'location':user['location'],
                  'description':user['description'],
                  'url':user['url'],
                  'followers_count':user['followers_count'],
                  'friends_count':user['friends_count'],
                  'listed_count':user['listed_count'],
                  'created_at':user['created_at'],
                  'favourites_count':user['favourites_count'],
                  'geo_enabled':user['geo_enabled'],
                  'verified':user['verified'],
                  'statuses_count':user['statuses_count'],
                  'lang':user['lang'],
                  'contributors_enabled':user['contributors_enabled']
                 }
        df_users = df_users.append(d_user,ignore_index=True)

def store_tweet(tweet):
    global df_tweets
    text = re.sub(r"http\S+", "", tweet['full_text'])
    df_tweets = df_tweets.append({'tweet_id':  tweet['id_str'],
            'created_at': tweet['created_at'],
            'text': text,
            'truncated': tweet['truncated'],
            'source': tweet['source'],
            'in_reply_to_status_id': tweet['in_reply_to_status_id_str'],
            'in_reply_to_user_id': tweet['in_reply_to_user_id_str'],
            'in_reply_to_screen_name': tweet['in_reply_to_screen_name'],
            'user_id': tweet['user']['id_str'],
            'geo': tweet['geo'],
            'coordinates': tweet['coordinates'],
            'place': tweet['place'],
            'contributors': tweet['contributors'],
            'is_quote_status': tweet['is_quote_status'],
            'retweet_count': tweet['retweet_count'],
            'favorite_count': tweet['favorite_count'],
            'favorited': tweet['favorited'],
            'retweeted': tweet['retweeted'],
            'lang': tweet['lang'] },ignore_index=True)
        
def store_retweet_user(tweet):
    global df_retweet_users
    if is_a_retweet(tweet):
        df_retweet_users = df_retweet_users.append({'user_id':tweet['user']['id_str'],
                                                    'original_user_id':tweet['retweeted_status']['user']['id_str'],
                                                    'original_tweet_id':tweet['retweeted_status']['id_str']}, ignore_index=True)
        
def store_influent_users(n):
    #n is the number of influent users return
    df_favorite_count = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).head()
    for i in range(n):
        d_influent_users = {'user_id':df_favorite_count[0], 
                            'name':[],
                            'retweet_count':[],
                            'favorite_count':[], 
                            'description':[] 
                           }
        

In [105]:
#### creation of the databases

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #https://stackoverflow.com/questions/40659212/futurewarning-elementwise-comparison-failed-returning-scalar-but-in-the-futur
import os
path_to_json = 'tweets/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for json_file in json_files:
    
    with open("tweets/"+json_file) as tweet:
        tweet = json.load(tweet)
        if is_a_retweet(tweet):
            store_retweet_user(tweet)
        else:
            store_tweet(tweet)
            store_users(tweet)
            store_hashtag(tweet)
            store_user_mentions(tweet)
        
        
        

In [109]:
df_tweets.head(10);


In [106]:
df_tweets['text'].iloc[0]

'Insupportable d\'entendre le #GiletJaune en roue libre sur #LCI dire, par exemple, que l\'élection d\'#EmmanuelMacron est "une erreur de l\'Histoire", et ajouter "j\'aimerais bien qu\'il bosse au lieu de faire du ski". Sans que la journaliste ne le pousse à s\'expliquer davantage...'

In [110]:
#primary keys are ok?
print(df_tweets.duplicated('tweet_id').any())
print(df_users.duplicated('user_id').any())


False
False


# Aggregations


## Top users

In [111]:
df_most_favorited_users = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_retweeted_users = df_tweets.groupby(['user_id'])['retweet_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_favorited_users['favorite_count'] = df_most_favorited_users['favorite_count'].astype('int32')
df_most_retweeted_users['retweet_count'] = df_most_retweeted_users['retweet_count'].astype('int32')

df_most_favorited_users['user_id'] = df_most_favorited_users['user_id'].astype(object)
df_most_retweeted_users['user_id'] = df_most_retweeted_users['user_id'].astype(object)



In [112]:
df_most_favorited_users_with_info = df_most_favorited_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [113]:
df_most_retweeted_users_with_info.head(15);

In [114]:
df_most_favorited_users_with_info.head(15);

In [115]:
df_most_retweeted_users_with_info.reset_index()
df_most_favorited_users_with_info.reset_index();

In [116]:
print(df_most_retweeted_users_with_info['name'])
print(df_most_favorited_users_with_info['name'])

user_id
472852289                        Brèves de presse
388972391                            Anasse Kazib
338955445                     de Cabarrus Thierry
2148731538                    Jean-Yves Le Gallou
742382030                         Gaëtan #FBPE  🌍
2470583048                         Charles Baudry
2653885316                         jean garrigues
203894274                         Cécile Ollivier
1028668294751039488            LesGiletsJaunes.fr
3688356795                     Jean-Pierre Bordes
840670437050875904             BleuBlancRouge4369
804606773848133633              Barthe Philippe 🐝
817189792769339392       Abbé Rézina 🇫🇷 ن 2⃣3⃣⛽️🚕
996698079180050433     ⚡️P'tweet Revue de Presse🗞
1064054889452634112      🇫🇷⭐️GiletsJauneRouen⭐️🇫🇷
Name: name, dtype: object
user_id
472852289                 Brèves de presse
338955445              de Cabarrus Thierry
388972391                     Anasse Kazib
742382030                  Gaëtan #FBPE  🌍
2653885316                  jean gar

In [117]:
df_tweets_by_user = df_tweets.groupby(['user_id']).size().to_frame().reset_index()
df_most_favorited_users_with_info = df_most_favorited_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [118]:
df_most_favorited_users_with_info;

In [119]:
top_influencers_fav = df_most_favorited_users_with_info[['screen_name','description', 'followers_count', 'friends_count','favorite_count',0]]
top_influencers_rt = df_most_retweeted_users_with_info[['screen_name','description', 'followers_count', 'friends_count','retweet_count',0]]
top_influencers_fav.columns = ["Name","Description","Number of followers","Number of friends","Number of favorites","Number of tweets"]
top_influencers_rt.columns = ["Name","Description","Number of followers","Number of friends","Number of retweets","Number of tweets"]
top_influencers_fav = top_influencers_fav.reset_index().drop(columns = ["user_id"])
top_influencers_rt = top_influencers_rt.reset_index().drop(columns = ["user_id"]);
top_influencers_fav['Number of followers'] = top_influencers_fav['Number of followers'].astype(int)
top_influencers_fav['Number of friends'] = top_influencers_fav['Number of friends'].astype(int)
top_influencers_rt['Number of followers'] = top_influencers_rt['Number of followers'].astype(int)
top_influencers_rt['Number of friends'] = top_influencers_rt['Number of friends'].astype(int)

In [120]:
#add ratios
top_influencers_fav.loc[:,'Like Tweet Ratio'] = top_influencers_fav.loc[:,"Number of favorites"]/top_influencers_fav.loc[:,"Number of tweets"]
top_influencers_rt.loc[:,'Retweet Tweet Ratio'] = top_influencers_rt.loc[:,"Number of retweets"]/top_influencers_rt.loc[:,"Number of tweets"]

In [123]:
top_influencers_fav_names = top_influencers_fav['Name'].values
top_influencers_rt_names = top_influencers_rt['Name'].values
print(top_influencers_rt_names)

['Brevesdepresse' 'AnasseKazib' 'tcabarrus' 'jylgallou' 'gaetan34070'
 'CharlesBaudry' 'jeangarr78' 'cecileollivier' 'GiletsJaunesFr'
 'jpbordes4873' 'BBR4369' 'philippe_barthe' 'Abbe_Rezina' 'P_tweet_presse'
 'dms768']


## Top hashtags for this input hashtags and by users

In [124]:
top_influencers_fav.style.set_properties(**{'text-align': 'left'});

In [125]:
df_tweets_hashtags = df_hashtags.set_index('tweet_id').join(df_tweets.set_index('tweet_id'))

In [126]:
df_tw_hash_user = df_tweets_hashtags.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))

In [127]:
#df_aggretated = df_tw_hash_user.groupby(['user_id','hashtag']).size().reset_index()
df_aggretated = df_tw_hash_user.groupby(['name','hashtag']).size().reset_index()

In [128]:
df_aggretated.columns = ["Name", "#", "Used"]
df_aggretated;


In [129]:
#useful encapsulation, aint it?
def get_tweet_by_userid(user_id):
    df_tweets[df_tweets['user_id'] == 'user_id']

def get_tweet_by_username(name):
    df_tweets_user_info = df_tweets.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))
    df_tweets_user_info = df_tweets_user_info[df_tweets_user_info['name'] == 'LesGiletsJaunes.fr']
    df_tweets_user_info = df_tweets_user_info[['tweet_id','text','created_at', 'favorite_count', 'retweet_count']]
    return df_tweets_user_info

def get_related_hashtags_by_username(name):
    d_name = df_aggretated[df_aggretated['Name'] == name][['#',"Used"]].sort_values("Used", ascending = False)
    return d_name

In [130]:
get_tweet_by_username('LesGiletsJaunes.fr')


,tweet_id,text,created_at,favorite_count,retweet_count
user_id,,,,,
1028668294751039488,1107771021665492998,Vous demandez une augmentation de 200 balles par mois ?\nPas de problème ...\n#GiletJaune #GiletsJaunes #YellowVests #SMIC \n#assembleenationale #senat #Europeennes2019 \n#droitsdelhomme #Amnestyinternational #ViolencesPolicieres #ONU\n#France2 #BFM #RTFrance #LCI #Cnews #CNN #CBSNews,Mon Mar 18 22:29:27 +0000 2019,52.0,47.0
1028668294751039488,1107783190708658176,"Agence BFM ( Benalla Fouille Mer ... ) :\nUn bandeau sur l’œil , comme Jean-Marie Le Pen ! ... le bras tendu !\nL'appartenance à la mouvance néo-extrême ne fait plus aucun doute !\nIls sont tous complices !\n#GiletJaune #GiletsJaunes #facknews d'état !",Mon Mar 18 23:17:48 +0000 2019,43.0,17.0
1028668294751039488,1107790134890676225,"Didier Maïsto : Retour sur l’Acte 18, décisions du Premier ministre et intervention dans ce live de Bruno Pomart, ancien instructeur du RAID.\n\n#GiletJaune #GiletsJaunes #assembleenationale #senat #Europeennes2019 \n#droitsdelhomme #ViolencesPolicieres #ONU",Mon Mar 18 23:45:24 +0000 2019,7.0,8.0


In [131]:
#output 
for name in top_influencers_rt_names:
    df_rel = get_related_hashtags_by_username(name)
    print(name)
    df_rel.to_csv('top_hashtags_by_'+name+'.csv',index=False)
    df_rel=[]

Brevesdepresse
AnasseKazib
tcabarrus
jylgallou
gaetan34070
CharlesBaudry
jeangarr78
cecileollivier
GiletsJaunesFr
jpbordes4873
BBR4369
philippe_barthe
Abbe_Rezina
P_tweet_presse
dms768


In [132]:
df_tweets['created_at'].values;


# NLP


In [133]:
import spacy
import fr_core_news_sm
nlp = fr_core_news_sm.load()

In [100]:
doc = nlp("Ceci est une phrase")

In [101]:
doc

Ceci est une phrase